# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [14]:
import Pkg; 
# Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
# using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

### Récupération des données

In [15]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [16]:
function TestsSondabilite_LP(x, Upper_Bound, listUB, quantity, listobjs, profit)
    TA, TO, TR = false, false, false
    # Test d'Admissibilité
    if (quantity < 0)
        TA = true
    end
    # Test de Résolution
    if (length(listobjs) == length(x))
        TR = true
        Bestsol = x
    end
    # Test d'Optimalité
    for i in 1:length(listUB)
        if (Upper_Bound < listUB[i])
            TO = true
        end
    end
    BestProfit = profit
    return TA, TO, TR, BestProfit
end

TestsSondabilite_LP (generic function with 1 method)

## Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [17]:

function SeparerNoeud_lexicographic_depthfirst!(listobjs, listvals, n)
    i, obj = 1, 0
    while((i <= n) && (obj==0))
        if(!(i in listobjs))
            obj=i
        end
        i+=1
    end
    
    println("\nbranch on object ", obj, "\n")

    # depthfirst exploration strategy: the node selected will be the most left of the child-nodes just created
    push!(listobjs,obj) #save the identity of the object selected for branching
    push!(listvals,1.0) #save the node selected, identified by the value assigned to the variable/object chosen
end

function ExplorerAutreNoeud!(listUB, listres, listprevious, listlistobjs)
    #  Chercher le noeud dans listUB ayant la meilleure upper_bound
    UB_temp = 0
    index = 0
    for i in 1:length(listUB)
        if (listUB[i] > UB_temp)
            UB_temp = listUB[i]
            index = i
        end
    end
    #  Mise à jour de x et de previous
    x = listres[index]
    previous = listprevious[index]
    listobjs = listlistobjs[index]
    # Mise à jour de la liste des noeuds et des upper_bounds
    deleteat!(listUB, index)
    deleteat!(listres, index)
    deleteat!(listprevious, index)
    deleteat!(listlistobjs, index)

    return x, listUB, listres, listprevious, listlistobjs, listobjs, previous
end

ExplorerAutreNoeud! (generic function with 1 method)

###  Création de la relaxation linéaire (= modèle associé au noeud 0): <span style="color:red"> SECTION A SUPPRIMER !!!! </span>

<span style="color:red"> Cette section est à commenter/supprimer et remplacer par vos propres calculs de bornes supérieures et autres, par exemple basées sur les bornes 1 et 2 vues en cours, ou d'autres calculs de bornes de votre choix/conception validés au préalable par votre encadrant/e de TP </span>

In [18]:
# Pkg.add("Clp");
# Pkg.add("JuMP");
# using JuMP, Clp

function set_UB_M1(x, capacity, price, weight, listeobj, obj_chosen)
    max = 0
    new_capacity = capacity - sum(x[i]*weight[i] for i in 1:length(x))
    for i in 1:length(listeobj)
        if max <= price[listeobj[i]]/weight[listeobj[i]]
            max = price[listeobj[i]]/weight[listeobj[i]]
        end
    end
    if max <= price[obj_chosen]/weight[obj_chosen]
        max = price[obj_chosen]/weight[obj_chosen]
    end
    return new_capacity*max
end

function set_UB_M2(x, price, weight, listeobj, obj_chosen, capacity, previous, add)
    res = 0
    new_capacity = capacity - sum(x[i]*weight[i] for i in 1:length(x))
    if add == true && previous != -1
        new_capacity = new_capacity - weight[previous]
        if new_capacity < 0
            res = -1
        else
            res = res + price[previous]
        end
    end
    for i in 1:length(listeobj)
        res = res + price[listeobj[i]]*x[listeobj[i]]
    end
    res = res + price[obj_chosen]*new_capacity/weight[obj_chosen]
    return res
end

function MajModele!(x, listUB, listres, listprevious, listlistobjs, capacity, price, weight, ratio, listeobj, previous, option)

    feasable = true
    ratio_max = 0
    obj_chosen = -1

    # Choose the object with the highest ratio
    for i in 1:length(ratio)
        if !(i in listeobj)
            if ratio[i] > ratio_max
                ratio_max = ratio[i]
                obj_chosen = i
            end
        end
    end
    bnd = -1

    #MAJ de x et de listeobj
    if (obj_chosen != -1)
        bnd1 = -1
        bnd0 = -1
        if (option == "Bound1")
            bnd = set_UB_M1(x, capacity, price, weight, listeobj, obj_chosen)
        elseif (option == "Bound2")
            bnd1 = set_UB_M2(x, price, weight, listeobj, obj_chosen, capacity, previous, true)
            # println("bnd1 = ", bnd1)

            bnd0 = set_UB_M2(x, price, weight, listeobj, obj_chosen, capacity, previous, false)
            # println("bnd0 = ", bnd0)
        end

        # push!(listres, x)

        # maj x et tout le Reste 
        if (bnd1 > bnd0)
            # ajouter bnd0 à liste bounds
            push!(listUB, bnd0)
            #  ajouter x à la liste des noeuds à traiter
            copy_x = copy(x)
            listres = push!(listres, copy_x)
            
            x[previous] = 1
            bnd = bnd1
        else
            # ajouter bnd1 à liste bounds
            push!(listUB, bnd1)
            #  Modier x avant de l'ajouter à la liste des noeuds à traiter
            x[previous] = 1
            copy_x = copy(x)
            push!(listres, copy_x)

            x[previous] = 0
            bnd = bnd0
        end
        copy_listobjs = copy(listeobj)
        push!(listlistobjs, copy_listobjs)
        push!(listeobj, obj_chosen)
        push!(listprevious, previous)

    else
        feasable = false
    end

    return bnd, x, listUB, listres, listprevious, listlistobjs, listeobj, feasable, obj_chosen
end

function solveProblem(x, capacity, constraints, objective)

    quantity = capacity - constraints(x)
    profit = objective(x)

    return profit, quantity
end

solveProblem (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [22]:
function SolveKnapInstance(filename)

    price, weight, capacity = readKnaptxtInstance(filename)
    n = length(price)

    # Définir x, contraintes et fonction objectif
    x = zeros(n)
    # contrainte 1: somme de Weight * x <= capacity
    # contrainte 2: x = {0 ou 1}
    constraints(y) = sum(weight[i]*y[i] for i in 1:n)
    # fonction objectif: x*price (max)
    objective(y) = sum(price[i]*y[i] for i in 1:n)
    # Calculer le rapport price/weight
    ratio = zeros(n)
    for i in 1:n
        ratio[i] = price[i]/weight[i]
    end

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listUB=[]
    listres=[]
    listnodes=[]
    listprevious=[]
    listlistobjs=[]

    BestProfit=-1
    Bestsol=[]

    current_node_number=0
    stop = false
    obj_chosen = -1
    ratio_max = 0

    for i in 1:length(ratio)
        if ratio[i] > ratio_max
            ratio_max = ratio[i]
            obj_chosen = i
        end
    end

    previous = obj_chosen
    Upper_Bound = price[obj_chosen]/weight[obj_chosen]*capacity
    push!(listobjs, obj_chosen)

    while(!stop)

        println("\nNode number ", current_node_number, ": \n---------------\n")

        Upper_Bound, x, listUB, listres, listprevious, listlistobjs, listobjs, feasable, previous = MajModele!(x, listUB, listres, listprevious, listlistobjs, capacity, price, weight, ratio, listobjs, previous, "Bound2")
        println("listobjs = ", listobjs)
        println("The new Upper bound is ", Upper_Bound)

        # create a new node (choose an object and compute the profit value)
        profit, quantity = solveProblem(x, capacity, constraints, objective)

        TA, TO, TR, BestProfit = TestsSondabilite_LP(x, Upper_Bound, listUB, quantity, listobjs, profit)

        # is_node_sondable = TA || TO || TR
        not_result = TA || TO

        if(not_result)
            println("blabla")
            # Chercher la UB max
            UB_temp = 0
            index = 0
            for i in 1:length(listUB)
                if listUB[i] > UB_temp
                    UB_temp = listUB[i]
                    index = i
                end
            end
            x = listres[index]
            previous = listprevious[index]
            listobjs = listlistobjs[index]
            deleteat!(listres, index)
            deleteat!(listUB, index)
            deleteat!(listprevious, index)
            deleteat!(listlistobjs, index)
        elseif (TR)
            stop = true
        end
        Bestsol = x
        current_node_number = current_node_number + 1
    end

    println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)

    return BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes

end


SolveKnapInstance (generic function with 1 method)

### Affichage du résultat final

In [23]:
BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = SolveKnapInstance("instancesETU/KNAPnewformat/test.opb.txt")
println("\n******\n\nOptimal value = ", BestProfit, "\n\nOptimal x=", Bestsol)
# graphplot(trParentnodes, trChildnodes, names=trNamenodes, method=:tree)


Node number 0: 
---------------

listobjs = Any[2, 1]
The new Upper bound is 76.0
x = [0.0, 1.0, 0.0, 0.0]

Node number 1: 
---------------

listobjs = Any[2, 1, 4]
The new Upper bound is 70.0
x = [0.0, 1.0, 0.0, 0.0]

Node number 2: 
---------------

listobjs = Any[2, 1, 4, 3]
The new Upper bound is 69.0
x = [0.0, 1.0, 0.0, 1.0]

******

Optimal value = 65.0

Optimal x=Any[]

******

Optimal value = 65.0

Optimal x=Any[]
